In [1]:
# Standard library imports
from datetime import datetime
import os

# Third-party library imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm  # Visualize loop progress
import xlwings as xw # dynamic Excel interaction
from lightweight_charts import Chart

# Set pandas display options for better data frame visualization
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Segreti
from dotenv import load_dotenv
load_dotenv()

# API
import requests
import json

| Gap-Up Screener |
|--|
TradingView Data (import)

In [2]:
# today's date string for file naming automation
today_date_str = datetime.now().strftime("_%Y-%m-%d.csv")
# today_date_str = '_2024-03-07.csv'

print(today_date_str)

_2024-03-11.csv


In [3]:
# concat base file name with today's date string
filename = f"trading_view_raw_data/tv_screen_gap-up{today_date_str}"
# read in trading view raw data
trading_view_raw_df = pd.read_csv(filename)

print(f'{len(trading_view_raw_df.index)} U.S. Stocks \n$(pre-market change) > $0.00 USD')
trading_view_raw_df.head(5)

1176 U.S. Stocks 
$(pre-market change) > $0.00 USD


,Symbol,Description,Exchange,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Sector,Relative Volume at Time,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 day,Relative Volume 1 week,Relative Volume 1 month,New high 1 month,New high 1 month - Currency,New high 3 months,New high 3 months - Currency,New high 6 months,New high 6 months - Currency,New high 52 weeks,New high 52 weeks - Currency,New high All Time,New high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower"
0,AAPL,Apple Inc.,NASDAQ,2636392253146,USD,170.73,USD,171.00,USD,2.04,USD,1.194869,0.158144,15427909638.6,76266505,393261729,645060,7.476463e+07,61053401.53333382,56388274.39999982,2.817568,1.776185,1.568710,171.123333,26.56449354,Electronic Technology,0.87238385,1.06712,1.2041771,1.2891104,4.699136,4.775449,3.102821,2.131078,1.035981,0.615886,0.802548,1.064432,1.52634414,0.39175434,189.990,USD,199.62,USD,199.62,USD,199.62,USD,199.620000,USD,171.345,USD,171.4700,USD,171.7328,USD,171.7328,USD,171.7328,USD,173.700,USD,173.700,USD,176.900,USD,180.53,USD,2380652.17391304,USD,170.95800,171.066875,171.1250,171.760969,171.452370,171.162730,170.873090,174.034678,172.273370,170.512062
1,GOOG,Alphabet Inc.,NASDAQ,1688542996689,USD,136.29,USD,136.13,USD,0.75,USD,0.550297,-0.117397,5277750176,26494430,145795972,108394,2.963381e+07,25350156.93333337,22350469.90000001,3.104599,2.463250,1.922458,136.691667,23.47722731,Technology Services,0.70494811,1.021201,1.1586931,1.0489115,5.942181,5.836621,3.725545,2.159699,1.101765,0.591959,0.599773,0.931711,1.4754574,0.36115306,150.695,USD,155.20,USD,155.20,USD,155.20,USD,155.200000,USD,136.565,USD,136.5850,USD,136.5850,USD,136.5850,USD,136.5850,USD,137.170,USD,138.985,USD,138.985,USD,140.00,USD,1683033.61059057,USD,136.41198,136.403112,136.3820,136.524877,136.565622,136.392495,136.219368,137.208154,136.648920,136.089686
2,GOOGL,Alphabet Inc.,NASDAQ,1688542996688.9998,USD,135.41,USD,135.25,USD,0.73,USD,0.539103,-0.118160,5871219472,39370316,208621307,150956,4.059745e+07,34980688.53333334,29774609.04444444,3.048507,2.495721,1.962727,135.831667,23.32563908,Technology Services,0.80240453,1.0305723,1.1459233,1.0442357,3.526613,4.881177,3.368677,1.973859,0.965876,0.539623,0.627396,1.019698,1.56655908,0.37931306,149.440,USD,153.78,USD,153.78,USD,153.78,USD,153.780000,USD,135.660,USD,135.6700,USD,135.6700,USD,135.6700,USD,135.6700,USD,136.235,USD,138.085,USD,138.085,USD,138.87,USD,1683033.61059057,USD,135.52400,135.498125,135.4680,135.587454,135.655508,135.469745,135.283982,136.249404,135.706345,135.163286
3,NVO,Novo Nordisk A/S,NYSE,594721816483.4071,USD,133.07,USD,134.05,USD,0.23,USD,0.172841,0.736455,3383416466.2

In [4]:
def categorize_market_cap(df):
    # convert 'Market capitalization' to numeric, coercing errors to NaN
    df['Market capitalization'] = pd.to_numeric(df['Market capitalization'], errors='coerce')
    # define conditions for market cap categories
    conditions = [
        # Titans
        (df['Market capitalization'] >= 200000000000),  # 200 billion and above
        #Large Cap
        (df['Market capitalization'] >= 10000000000) & (df['Market capitalization'] < 200000000000),  # 10 billion to 200 billion
        # Midlers
        (df['Market capitalization'] >= 2000000000) & (df['Market capitalization'] < 10000000000),  # 2 billion to 10 billion
        # Small Cap
        (df['Market capitalization'] >= 300000000) & (df['Market capitalization'] < 2000000000),  # 300 million to 2 billion
        # Micro Cap
        (df['Market capitalization'] > 50000000) & (df['Market capitalization'] < 300000000),  # 300 million and below (excluding 50 million and below)
        # Shrimp
        (df['Market capitalization'] <= 50000000)  # 50 million and below
    ]
    categories = ['Titans', 'Large caps', 'Midlers', 'Small caps', 'Micro caps', 'Shrimp']
    # use np.select to assign categories based on conditions
    df['marketCapType'] = np.select(conditions, categories, default='Undefined')
    
    return df


In [5]:
# EXECUTE
trading_view_raw_df = categorize_market_cap(trading_view_raw_df)
trading_view_raw_df.head(5)

,Symbol,Description,Exchange,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Sector,Relative Volume at Time,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 day,Relative Volume 1 week,Relative Volume 1 month,New high 1 month,New high 1 month - Currency,New high 3 months,New high 3 months - Currency,New high 6 months,New high 6 months - Currency,New high 52 weeks,New high 52 weeks - Currency,New high All Time,New high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower",marketCapType
0,AAPL,Apple Inc.,NASDAQ,2.636392e+12,USD,170.73,USD,171.00,USD,2.04,USD,1.194869,0.158144,15427909638.6,76266505,393261729,645060,7.476463e+07,61053401.53333382,56388274.39999982,2.817568,1.776185,1.568710,171.123333,26.56449354,Electronic Technology,0.87238385,1.06712,1.2041771,1.2891104,4.699136,4.775449,3.102821,2.131078,1.035981,0.615886,0.802548,1.064432,1.52634414,0.39175434,189.990,USD,199.62,USD,199.62,USD,199.62,USD,199.620000,USD,171.345,USD,171.4700,USD,171.7328,USD,171.7328,USD,171.7328,USD,173.700,USD,173.700,USD,176.900,USD,180.53,USD,2380652.17391304,USD,170.95800,171.066875,171.1250,171.760969,171.452370,171.162730,170.873090,174.034678,172.273370,170.512062,Titans
1,GOOG,Alphabet Inc.,NASDAQ,1.688543e+12,USD,136.29,USD,136.13,USD,0.75,USD,0.550297,-0.117397,5277750176,26494430,145795972,108394,2.963381e+07,25350156.93333337,22350469.90000001,3.104599,2.463250,1.922458,136.691667,23.47722731,Technology Services,0.70494811,1.021201,1.1586931,1.0489115,5.942181,5.836621,3.725545,2.159699,1.101765,0.591959,0.599773,0.931711,1.4754574,0.36115306,150.695,USD,155.20,USD,155.20,USD,155.20,USD,155.200000,USD,136.565,USD,136.5850,USD,136.5850,USD,136.5850,USD,136.5850,USD,137.170,USD,138.985,USD,138.985,USD,140.00,USD,1683033.61059057,USD,136.41198,136.403112,136.3820,136.524877,136.565622,136.392495,136.219368,137.208154,136.648920,136.089686,Titans
2,GOOGL,Alphabet Inc.,NASDAQ,1.688543e+12,USD,135.41,USD,135.25,USD,0.73,USD,0.539103,-0.118160,5871219472,39370316,208621307,150956,4.059745e+07,34980688.53333334,29774609.04444444,3.048507,2.495721,1.962727,135.831667,23.32563908,Technology Services,0.80240453,1.0305723,1.1459233,1.0442357,3.526613,4.881177,3.368677,1.973859,0.965876,0.539623,0.627396,1.019698,1.56655908,0.37931306,149.440,USD,153.78,USD,153.78,USD,153.78,USD,153.780000,USD,135.660,USD,135.6700,USD,135.6700,USD,135.6700,USD,135.6700,USD,136.235,USD,138.085,USD,138.085,USD,138.87,USD,1683033.61059057,USD,135.52400,135.498125,135.4680,135.587454,135.655508,135.469745,135.283982,136.249404,135.706345,135.163286,Titans
3,NVO,Novo Nordisk A/S,NYSE,5.947218e+11,USD,133.07,USD,134.05,USD,0.23,USD,0.172841

In [6]:
# drop NaNs
tv_df = trading_view_raw_df.dropna()
print(f'NaNs = {tv_df.isna().sum().sum()}')

# confirm no NaNs
if tv_df.isna().sum().sum() == 0:
    nan_count = tv_df.isna().sum().sum()
    print(f"There are zero {nan_count} NaNs in DataFrame! Nice work!")
else:
    print(f"There are {tv_df.isna().sum().sum()} NaNs in DataFrame, check your code dumb dumb!")

NaNs = 0
There are zero 0 NaNs in DataFrame! Nice work!


In [7]:
# view the raw counts and percentages of each market cap type
market_cap_type_df = trading_view_raw_df['marketCapType'].value_counts().reset_index()
market_cap_type_df.columns = ['marketCapType', 'count']  # Rename columns for clarity
# calculate percentage -> add new columne to dataframe
market_cap_type_df['percentage'] = (market_cap_type_df['count'] / market_cap_type_df['count'].sum()) * 100

print(f'There are still the same {len(trading_view_raw_df.index)} U.S. Stocks in the dataframe \n$(pre-market change) > $0.00 USD')
market_cap_type_df

There are still the same 1176 U.S. Stocks in the dataframe 
$(pre-market change) > $0.00 USD


,marketCapType,count,percentage
0,Small caps,284,24.149660
1,Shrimp,274,23.299320
2,Micro caps,247,21.003401
3,Midlers,185,15.731293
4,Large caps,151,12.840136
5,Titans,21,1.785714
6,Undefined,14,1.190476


In [8]:
# print length of df
print(f'{len(tv_df.index)} U.S. Stocks \n$(pre-market change) > $0.00 USD')
# print sample head
tv_df.sample(5)

1162 U.S. Stocks 
$(pre-market change) > $0.00 USD


,Symbol,Description,Exchange,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Sector,Relative Volume at Time,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 day,Relative Volume 1 week,Relative Volume 1 month,New high 1 month,New high 1 month - Currency,New high 3 months,New high 3 months - Currency,New high 6 months,New high 6 months - Currency,New high 52 weeks,New high 52 weeks - Currency,New high All Time,New high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower",marketCapType
462,HIMX,"Himax Technologies, Inc.",NASDAQ,1.004639e+09,USD,5.76,USD,5.86,USD,0.0200,USD,0.347222,1.736111,174416525,671519,4988233,3800,865366.0,846536.5,789406.18888889,3.391304,4.545896,3.572005,5.818333,19.78021978,Electronic Technology,0.54381825,1.2090609,1.918666,2.0699725,2.139920,3.459871,2.627216,2.001611,0.797325,0.418963,0.519255,0.777978,1.42853191,0.3537698,6.03,USD,6.39,USD,6.39,USD,8.20,USD,17.880000,USD,5.770,USD,5.7700,USD,5.770,USD,5.770,USD,5.770,USD,5.800,USD,5.9450,USD,6.0300,USD,6.0300,USD,-,-,5.7680,5.766875,5.764000,5.764615,5.773274,5.764000,5.754726,5.782059,5.767750,5.753441,Small caps
13,KO,Coca-Cola Company (The),NYSE,2.566774e+11,USD,59.52,USD,59.77,USD,0.1602,USD,0.269153,0.420027,4284855943.540001,13239013,61764889,31157,11918882.4,14317435.33333334,14123369.8,1.390538,1.142964,1.202094,59.426667,24.1079023,Consumer Non-Durables,0.8516902,0.22005098,0.46492413,0.5911441,4.967438,6.422605,5.334688,2.770047,1.043896,0.636299,0.685865,1.107286,0.99331106,0.24706512,61.62,USD,61.62,USD,61.62,USD,64.99,USD,67.200000,USD,59.630,USD,59.6300,USD,59.650,USD,59.650,USD,59.650,USD,59.680,USD,59.7900,USD,60.1100,USD,60.1100,USD,579405.81542352,USD,59.5560,59.570625,59.578000,59.603077,59.623444,59.575545,59.527646,59.682357,59.607545,59.532733,Titans
950,MMV,MultiMetaVerse Holdings Limited,NASDAQ,2.809158e+07,USD,0.85,USD,0.92,USD,0.0500,USD,5.882353,8.235294,7028179.074,30797,2579519,1300,270930.2,129981.03333333,57609.76666667,12.357920,21.290909,16.805685,0.868733,-,Consumer Services,-,2.9934216,0.17795102,0.17795102,2.706360,0.837553,0.821410,0.253105,0.151394,0.067419,0.181091,0.114058,16.37396866,0.22828015,1.43,USD,1.43,USD,1.59,USD,4.00,USD,18.600010,USD,0.875,USD,0.9000,USD,0.900,USD,0.900,USD,0.900,USD,0.900,USD,0.9292,USD,1.1699,USD,1.1699,USD,0,USD,0.8717,0.861280,0.864329,0.869165,0.925914,0.871382,0.816849,0.932702,0.875987,0.819273,Shrimp
253,VLY,Valley National Bancorp,NASDAQ,4.050722e+09,USD,7.96,USD,8.02,USD,0.0400,USD,0.502513,0.753769,428078641.965,5748278,67112780,3220,9183366.0,8471122.36666667,5059961.03333333,5.689001,9.233810,5.393865,8.076667,8.40371622,Finance,0.40587905,1.

In [9]:
# drop undefined
tv_df = tv_df[tv_df['marketCapType'] != 'Undefined']

market_cap_type_df2 = tv_df['marketCapType'].value_counts().reset_index()
market_cap_type_df2.columns = ['marketCapType', 'count']  # Rename columns for clarity

# calculate percentage for each market cap type and add it to the DataFrame
market_cap_type_df2['percentage'] = (market_cap_type_df2['count'] / market_cap_type_df2['count'].sum()) * 100

market_cap_type_df2
print(market_cap_type_df2)


  marketCapType  count  percentage
0    Small caps    284   24.440620
1        Shrimp    274   23.580034
2    Micro caps    247   21.256454
3       Midlers    185   15.920826
4    Large caps    151   12.994836
5        Titans     21    1.807229


In [10]:
# 1/3 #### CONFIG DATA SECTION ####
criteria_config = {
    "Titans": {
        "pre_market_change_pct_threshold": 0.002,  # Reduced to 0.2% for Titans
        "float_shares_outstanding_threshold": 1000000000,  # Increased to 1 billion shares
        "relative_volume_threshold": 1.2,  # Slightly more inclusive
        "relative_volume_at_time_threshold": 0.03,  # More inclusive
        "pre_market_gap_percentage_threshold": 0.001,  # Reduced to 0.1%
        "pre_market_vmap_drawdown_threshold": 0.003  # .3% drawdown from VWAP
    },
    "Large caps": {
        "pre_market_change_pct_threshold": 0.005,  # Reduced to 0.5% for Large caps
        "float_shares_outstanding_threshold": 200000000,  # Increased to 200 million shares
        "relative_volume_threshold": 1.3,  # Slightly more inclusive
        "relative_volume_at_time_threshold": 0.04,  # More inclusive
        "pre_market_gap_percentage_threshold": 0.005,  # Reduced to 0.5%
        "pre_market_vmap_drawdown_threshold": 0.004  # .4% drawdown from VWAP
    },
    "Midlers": {
        "pre_market_change_pct_threshold": 0.02,  # 2% for Mid caps
        "float_shares_outstanding_threshold": 50000000,  # 50 million shares
        "relative_volume_threshold": 1.3,
        "relative_volume_at_time_threshold": .05,
        "pre_market_gap_percentage_threshold": .02,
        "pre_market_vmap_drawdown_threshold": 0.005  # .5% drawdown from VWAP
    },
    "Small caps": {
        "pre_market_change_pct_threshold": 0.03,  # 3% for Small caps
        "float_shares_outstanding_threshold": 20000000,  # 20 million shares
        "relative_volume_threshold": 1.2,
        "relative_volume_at_time_threshold": .05,
        "pre_market_gap_percentage_threshold": .03,
        "pre_market_vmap_drawdown_threshold": 0.006  # 6% drawdown from VWAP
    },
    "Micro caps": {
        "pre_market_change_pct_threshold": 0.04,  # 4% for Micro caps
        "float_shares_outstanding_threshold": 5000000,  # 5 million shares
        "relative_volume_threshold": 1.1,
        "relative_volume_at_time_threshold": .05,
        "pre_market_gap_percentage_threshold": .04,
        "pre_market_vmap_drawdown_threshold": 0.007  # 7% drawdown from VWAP
    },
    "Shrimp": {
        "pre_market_change_pct_threshold": 0.05,  # 5% for Shrimp
        "float_shares_outstanding_threshold": 1000000,  # 1 million shares
        "relative_volume_threshold": 1.0,
        "relative_volume_at_time_threshold": .05,
        "pre_market_gap_percentage_threshold": .05,
        "pre_market_vmap_drawdown_threshold": 0.008  # 8% drawdown from VWAP
    }
}

# 2/3 #### CORE FUNCTIONS ####
def filter_by_pre_market_change(df, change_pct_threshold):
    """Filter stocks by pre-market change percentage."""
    return df[df['Pre-market Change %'] >= change_pct_threshold]

def filter_by_float_shares(df, float_shares_threshold):
    """Filter stocks by float shares outstanding."""
    # create a copy to safely modify without affecting the original DataFrame
    modified_df = df.copy()
    modified_df['Float shares outstanding'] = pd.to_numeric(modified_df['Float shares outstanding'], errors='coerce')
    return modified_df[modified_df['Float shares outstanding'] <= float_shares_threshold]

def filter_by_relative_volume(df, relative_volume_threshold):
    """Filter stocks by relative volume."""
    df['Relative Volume 1 day'] = pd.to_numeric(df['Relative Volume 1 day'], errors='coerce')
    return df[df['Relative Volume 1 day'] >= relative_volume_threshold]

def filter_by_relative_volume_at_time(df, relative_volume_at_time_threshold):
    """Filter stocks by relative volume at a specific time."""
    # Convert 'Relative Volume at Time' to float
    df['Relative Volume at Time'] = pd.to_numeric(df['Relative Volume at Time'], errors='coerce')
    return df[df['Relative Volume at Time'] >= relative_volume_at_time_threshold]

def filter_by_pre_market_gap_percentage(df, pre_market_gap_percentage_threshold):
    """Filter stocks by pre-market gap percentage."""
    return df[df['Pre-market Gap %'] >= pre_market_gap_percentage_threshold]

def filter_by_price_near_vwap(df, pre_market_vmap_drawdown_threshold):
    """
    Filter stocks where the price is near the VWAP, considering the specified drawdown threshold.
    """
    df_filtered = df.copy()
    df_filtered['Price'] = pd.to_numeric(df_filtered['Price'], errors='coerce')
    df_filtered['Volume Weighted Average Price 1 day'] = pd.to_numeric(df_filtered['Volume Weighted Average Price 1 day'], errors='coerce')
    
    # Calculate the acceptable minimum price based on the VWAP drawdown threshold
    df_filtered['Min Price from VWAP'] = df_filtered['Volume Weighted Average Price 1 day'] * (1 - pre_market_vmap_drawdown_threshold)
    
    # Apply the filter
    df_filtered = df_filtered[df_filtered['Price'] >= df_filtered['Min Price from VWAP']]
    
    return df_filtered.drop(columns=['Min Price from VWAP'])

"""SPECIAL FILTERS"""
def filter_by_volatility(df):
    """Filter stocks based on recent volatility being higher than weekly and monthly averages."""
    # Ensure data types are correct; convert to numeric if necessary
    df['Volatility 1 day'] = pd.to_numeric(df['Volatility 1 day'], errors='coerce')
    df['Volatility 1 week'] = pd.to_numeric(df['Volatility 1 week'], errors='coerce')
    df['Volatility 1 month'] = pd.to_numeric(df['Volatility 1 month'], errors='coerce')
    filtered_df = df[
        (df['Volatility 1 day'] >= df['Volatility 1 week']) & 
        (df['Volatility 1 day'] >= df['Volatility 1 month'])]
    return filtered_df

# 3/3 #### WRAPPER FUNCTION ####
def screen_stocks_by_category(df, category):
    """Screen stocks based on their market cap category."""
    # 3a. Get the configuration for the specified category
    config = criteria_config.get(category, {})
    pre_market_change_pct_threshold = config.get("pre_market_change_pct_threshold", 0)
    float_shares_threshold = config.get("float_shares_outstanding_threshold", float('inf'))
    relative_volume_threshold = config.get("relative_volume_threshold", 0)
    relative_volume_at_time_threshold = config.get("relative_volume_at_time_threshold", 0)
    pre_market_gap_percentage_threshold = config.get("pre_market_gap_percentage_threshold", 0)
    pre_market_vmap_drawdown_threshold = config.get("pre_market_vmap_drawdown_threshold", 0)
    # 3b. Apply the filters
    filtered_df = filter_by_pre_market_change(df, pre_market_change_pct_threshold)
    filtered_df = filter_by_float_shares(filtered_df, float_shares_threshold)
    filtered_df = filter_by_relative_volume(filtered_df, relative_volume_threshold)
    filtered_df = filter_by_relative_volume_at_time(filtered_df, relative_volume_at_time_threshold)
    filtered_df = filter_by_volatility(filtered_df) # SPECIAL FILTER
    # filtered_df = filter_by_price_above_vwap(filtered_df) # SPECIAL FILTER
    filtered_df = filter_by_pre_market_gap_percentage(filtered_df, pre_market_gap_percentage_threshold)
    filtered_df = filter_by_price_near_vwap(filtered_df, pre_market_vmap_drawdown_threshold)
  
    # 3c. Return the filtered DataFrame
    return filtered_df

In [11]:
#EXECUTE
tv_L2_df = pd.DataFrame()
categories = tv_df['marketCapType'].unique()

for category in categories:
    category_df = tv_df[tv_df['marketCapType'] == category]
    screened_df = screen_stocks_by_category(category_df, category)
    tv_L2_df = pd.concat([tv_L2_df, screened_df])

# Display results
print(f'Original DataFrame length: {len(tv_df.index)}')
print(f'Number of rows removed: {len(tv_df.index) - len(tv_L2_df.index)}')
print(f'NEW DataFrame length: {len(tv_L2_df.index)}')

# filter in descending order by market cap and then by pre-market change percentage
tv_L2_df = tv_L2_df.sort_values(by=['Market capitalization', 'Pre-market Change %'], ascending=[False, False])
# reset index and drop
tv_L2_df = tv_L2_df.reset_index(drop=True)

Original DataFrame length: 1162
Number of rows removed: 1155
NEW DataFrame length: 7


In [12]:
# create a breakdown of the market cap types
market_cap_type_df3 = tv_L2_df['marketCapType'].value_counts().reset_index()
market_cap_type_df3.columns = ['marketCapType', 'count']  # Rename columns for clarity
# calculate percentage for each market cap type and add it to the DataFrame
market_cap_type_df3['percentage'] = (market_cap_type_df3['count'] / market_cap_type_df3['count'].sum()) * 100

print('Returned Gap-Up Market Cap Type Groups:')
market_cap_type_df3

Returned Gap-Up Market Cap Type Groups:


,marketCapType,count,percentage
0,Small caps,3,42.857143
1,Large caps,2,28.571429
2,Midlers,1,14.285714
3,Shrimp,1,14.285714


In [13]:
# market cap type groups
print(f"Market Cap Groups:\n{tv_L2_df['marketCapType'].unique()}\n")

# TradingView column names
tv_raw_columns_list = tv_L2_df.columns.tolist()
print(f'TradingView Column Names (raw data):\n{tv_raw_columns_list}\n')

# stocks returned from the pre-market gap up screener
print(f'{len(tv_L2_df.index)} = U.S. stocks returned from the pre-market gap up screener.')

# create a list of 'Symbols' from the tv_L2_df
gap_up_stocks_list = tv_L2_df['Symbol']
gap_up_stocks_list = gap_up_stocks_list.tolist()
print(gap_up_stocks_list)

# FILTERD DATAFRAME WITH GAP-UP STOCK TARGETS
tv_L2_df


Market Cap Groups:
['Large caps' 'Midlers' 'Small caps' 'Shrimp']

TradingView Column Names (raw data):
['Symbol', 'Description', 'Exchange', 'Market capitalization', 'Market capitalization - Currency', 'Price', 'Price - Currency', 'Pre-market Open', 'Pre-market Open - Currency', 'Pre-market Change', 'Pre-market Change - Currency', 'Pre-market Change %', 'Pre-market Gap %', 'Float shares outstanding', 'Volume 1 day', 'Volume 1 week', 'Pre-market Volume', 'Average Volume 10 days', 'Average Volume 30 days', 'Average Volume 90 days', 'Volatility 1 day', 'Volatility 1 week', 'Volatility 1 month', 'Volume Weighted Average Price 1 day', 'Price to earnings ratio', 'Sector', 'Relative Volume at Time', 'Beta 1 year', 'Beta 3 years', 'Beta 5 years', 'Relative Volume 1 minute', 'Relative Volume 5 minutes', 'Relative Volume 15 minutes', 'Relative Volume 30 minutes', 'Relative Volume 1 hour', 'Relative Volume 2 hours', 'Relative Volume 4 hours', 'Relative Volume 1 day', 'Relative Volume 1 week', 'R

,Symbol,Description,Exchange,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Sector,Relative Volume at Time,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 day,Relative Volume 1 week,Relative Volume 1 month,New high 1 month,New high 1 month - Currency,New high 3 months,New high 3 months - Currency,New high 6 months,New high 6 months - Currency,New high 52 weeks,New high 52 weeks - Currency,New high All Time,New high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower",marketCapType
0,EDU,"New Oriental Education & Technology Group, Inc...",NYSE,1.497948e+10,USD,90.17,USD,90.47,USD,0.13,USD,0.144172,0.332705,1.654587e+08,2687256,9137894,3692,1524569.7,1768768.69999998,1785831.22222223,5.488575,3.932455,3.286652,89.710000,49.21137368,Consumer Services,1.490119,1.1611879,0.24545807,0.5030385,2.555885,6.406386,7.290278,2.120533,0.976995,0.602067,0.689005,1.893552,1.07552366,0.29500058,98.200,USD,98.20,USD,98.200,USD,98.20,USD,199.7400,USD,90.250,USD,90.25,USD,90.25,USD,90.25,USD,90.25,USD,90.36,USD,91.87,USD,98.20,USD,98.200,USD,54007.85500667,USD,90.15500,90.143750,89.99200,90.014231,90.274081,89.98081,89.687539,90.243335,89.940500,89.637665,Large caps
1,ONON,On Holding AG,NYSE,1.047365e+10,USD,33.09,USD,33.84,USD,0.16,USD,0.483530,2.266546,1.876031e+08,6576098,30306742,34748,5341704.4,4837171.2,5054020.62222222,3.581905,3.184276,3.365261,32.885000,118.68723099,Consumer Non-Durables,0.979479,1.6530961,2.179955,2.179955,2.480428,3.250074,3.452546,2.385260,1.209855,0.724526,0.673585,1.319468,1.33320761,0.36159538,35.580,USD,35.58,USD,35.580,USD,37.08,USD,55.8700,USD,33.100,USD,33.11,USD,33.11,USD,33.11,USD,33.11,USD,33.11,USD,33.18,USD,34.72,USD,35.125,USD,752262.56024297,USD,33.07398,33.066862,33.01400,32.953000,33.110527,33.01462,32.918713,33.086952,32.934450,32.781948,Large caps
2,MTN,"Vail Resorts, Inc.",NYSE,8.495413e+09,USD,223.76,USD,224.60,USD,0.24,USD,0.107258,0.375402,3.752368e+07,720663,3615720,1194,535531.6,400322.9,393717.48888889,3.635113,3.531393,2.202390,222.566667,41.53472055,Consumer Services,1.212317,1.1425825,0.7754122,1.1195556,5.844120,10.808708,8.242927,4.580450,1.703201,1.009830,0.718746,1.464385,2.06868201,0.48192066,236.920,USD,236.92,USD,254.780,USD,258.13,USD,376.2400,USD,224.365,USD,224.91,USD,225.47,USD,225.47,USD,225.47,USD,225.79,USD,225.79,USD,235.81,USD,235.810,USD,50869.08450704,USD,224.14100,224.090000,224.43300,224.837692,225.108001,224.38350,223.658999,225.762724,224.692750,223.622776,Midlers
3,AFYA,Afya Limited,NASDAQ,1.941000e+09,USD,20.71,USD,21.00,USD,0.29,USD,1.400290,1.400290,1.798361e+07,131873,355394,992,66157.6,89576.43333333,155568.8,4.214407,2.344296,2.535

| Gap-Up Screener |
|--|
IEX Cloud API (GET Request)

In [14]:
"""
A.
THIS IS ALSO GOING TO BE YOUR NEWS, CURRENT NEWS (like almost real-time (within 15-30mins))
-try to get foreign news where people are awake
-translation (internationalization) feature would be both advanced as well as practical
-all info about these companies so we start to learn what we are wokring with today
-maybe some fundamentals although less important for gap up? unless gap up turns into swing trade

B.
-need a closeness list. some rating system. avoid if a stock barely misses one or two filter metrics.
-so it's two part, 1-the filter closeness boolean as well as well has how close this miss was in terms of percentage and like an aggregate or miss difference -> THEN scoring system?

C.
-you are going to need also a -> 'chart_tv_df (this will be for the TradingView data)

"""

"\nA.\nTHIS IS ALSO GOING TO BE YOUR NEWS, CURRENT NEWS (like almost real-time (within 15-30mins))\n-try to get foreign news where people are awake\n-translation (internationalization) feature would be both advanced as well as practical\n-all info about these companies so we start to learn what we are wokring with today\n-maybe some fundamentals although less important for gap up? unless gap up turns into swing trade\n\nB.\n-need a closeness list. some rating system. avoid if a stock barely misses one or two filter metrics.\n-so it's two part, 1-the filter closeness boolean as well as well has how close this miss was in terms of percentage and like an aggregate or miss difference -> THEN scoring system?\n\nC.\n-you are going to need also a -> 'chart_tv_df (this will be for the TradingView data)\n\n"

In [15]:
"""NEWS AND THAT INTERNATIONALIZATION SHIT GOES HERE"""
info_news_df = pd.DataFrame(tv_L2_df, columns=['Symbol', 'Exchange', 'marketCapType'])

info_news_df

,Symbol,Exchange,marketCapType
0,EDU,NYSE,Large caps
1,ONON,NYSE,Large caps
2,MTN,NYSE,Midlers
3,AFYA,NASDAQ,Small caps
4,GRND,NYSE,Small caps
5,NMM,NYSE,Small caps
6,PRSO,NASDAQ,Shrimp


In [16]:
# # manual add list
# manual_add_list = ['DELL','PLTR','TSLA']
# # append list with manually added stocks
# gap_up_stocks_list.extend(manual_add_list)

# print(f'{manual_add_list} added manually\n')
# print(f'New list: {gap_up_stocks_list}')

In [17]:
#### IEX CLOUD API KEY -> VARIABLE ####
iex_api_key = os.getenv("IEX_API_KEY")

# set the list of symbols variable
symbols = ','.join(gap_up_stocks_list)

# SETTING THE DATE RANGE
"""
y - Example: range=1y -> returns (one year)
ytd - Example: range=ytd -> returns (year-to-date)
m - Example: range=6m returns (six months)
d - Example: range=5d returns (five days)
"""
data_range = '5m' # DATE RANGE (for historical stock price data)

url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbols}&types=chart&range={data_range}&token={iex_api_key}'

# Make the GET request
response = requests.get(url)

# Initialize a list to collect DataFrames for each symbol
dataframes_list = []

# Check if the request was successful
if response.status_code == 200:
    # Convert the response to JSON
    data = response.json()

    # Process and print the data with a progress bar
    for symbol in tqdm(gap_up_stocks_list, desc="Processing symbols"):
        if symbol in data:
            # Create a DataFrame for the current symbol's data
            symbol_df = pd.DataFrame(data[symbol]['chart'])
            # Add a 'Ticker' column to identify the data
            symbol_df['Ticker'] = symbol
            # Append the current symbol's DataFrame to the list
            dataframes_list.append(symbol_df)
            # print(f'Data for {symbol}:')
            # print(symbol_df.head(5))  # print the first few rows of the DataFrame for this symbol
else:
    print(f'Failed to fetch data: {response.status_code}')

# concatenate dataframes and create a combined list
historical_df = pd.concat(dataframes_list, ignore_index=True)


Processing symbols: 100%|██████████| 7/7 [00:00<00:00, 1314.36it/s]


In [18]:
# Set variables for historical data analysis
historical_length = len(historical_df.index)
unique_stocks = historical_df['Ticker'].nunique()
start_date = historical_df['date'].min()
end_date = historical_df['date'].max()

print(f"START date: {start_date}")
print(f"END date: {end_date}")
print(f'{historical_df["date"].nunique()} days of historical stock data.\n')
print(f"{historical_length} = Length of dataframe")
print(f"{unique_stocks} = Unique stocks in dataframe")

# Entries per stock
days_per_stock = historical_length / unique_stocks
print(f"{days_per_stock} = Date entries per stock (MATH check -> should be the same as the unique )\n")

# View the stocks
print(f'Unique stock ticker symbols: {gap_up_stocks_list}')
historical_cols_list = historical_df.columns.tolist()
print(f'Columns: {historical_cols_list}')

# convert df to .csv
# historical_df.to_csv(f'historical_stock_data{today_date_str}.csv', index=False)

# Display the first few rows of the dataframe
historical_df.head()

START date: 2023-10-11
END date: 2024-03-08
103 days of historical stock data.

721 = Length of dataframe
7 = Unique stocks in dataframe
103.0 = Date entries per stock (MATH check -> should be the same as the unique )

Unique stock ticker symbols: ['EDU', 'ONON', 'MTN', 'AFYA', 'GRND', 'NMM', 'PRSO']
Columns: ['close', 'high', 'low', 'open', 'priceDate', 'symbol', 'volume', 'id', 'key', 'subkey', 'date', 'updated', 'changeOverTime', 'marketChangeOverTime', 'uOpen', 'uClose', 'uHigh', 'uLow', 'uVolume', 'fOpen', 'fClose', 'fHigh', 'fLow', 'fVolume', 'label', 'change', 'changePercent', 'Ticker']


,close,high,low,open,priceDate,symbol,volume,id,key,subkey,date,updated,changeOverTime,marketChangeOverTime,uOpen,uClose,uHigh,uLow,uVolume,fOpen,fClose,fHigh,fLow,fVolume,label,change,changePercent,Ticker
0,64.43,65.920,63.2001,63.36,2023-10-11,EDU,1683723.0,HISTORICAL_PRICES,EDU,,2023-10-11,1697072463000,0.000000,0.000000,63.36,64.43,65.920,63.2001,1683723,63.36,64.43,65.920,63.2001,1683723.0,"Oct 11, 23",0.00,0.0000,EDU
1,62.65,65.380,62.2400,64.86,2023-10-12,EDU,1185528.0,HISTORICAL_PRICES,EDU,,2023-10-12,1697164226000,-0.027627,-0.027627,64.86,62.65,65.380,62.2400,1185528,64.86,62.65,65.380,62.2400,1185528.0,"Oct 12, 23",-1.78,-0.0276,EDU
2,62.37,63.170,61.9000,61.97,2023-10-13,EDU,669074.0,HISTORICAL_PRICES,EDU,,2023-10-13,1697247056000,-0.031973,-0.031973,61.97,62.37,63.170,61.9000,669074,61.97,62.37,63.170,61.9000,669074.0,"Oct 13, 23",-0.28,-0.0045,EDU
3,62.82,63.365,61.2300,61.42,2023-10-16,EDU,622083.0,HISTORICAL_PRICES,EDU,,2023-10-16,1697504424000,-0.024988,-0.024988,61.42,62.82,63.365,61.2300,622083,61.42,62.82,63.365,61.2300,622083.0,"Oct 16, 23",0.45,0.0072,EDU
4,61.99,63.000,61.6600,62.14,2023-10-17,EDU,535338.0,HISTORICAL_PRICES,EDU,,2023-10-17,1697590887000,-0.037871,-0.037871,62.14,61.99,63.000,61.6600,535338,62.14,61.99,63.000,61.6600,535338.0,"Oct 17, 23",-0.83,-0.0132,EDU


In [19]:
#### DEV - DATA ORGANIZATION ####
# convert df to .csv
# historical_df.to_csv(f'/Users/sudz4/Desktop/FINANCIAL-SYSTEMS/dark-trader/trading_view_raw_data/historical_stock_data{today_date_str}.csv', index=False)

In [20]:
# Ensure the dataframe is sorted by date if it's not already
historical_sorted_df = historical_df.sort_values(by='date')

# Select mandatory columns for a historical price bar chart + volume
# and explicitly copy the dataframe to ensure it's not a view but a new object
chart_historical_df = historical_sorted_df[[
    'symbol',   # *MANDATORY
    'date',     # *MANDATORY
    'close',    # *MANDATORY
    'high',     # *MANDATORY
    'low',      # *MANDATORY
    'open',     # *MANDATORY
    'volume'    # *MANDATORY
]].copy()

# Group by 'symbol' and get the last occurrence
last_date_df = chart_historical_df.groupby('symbol').last().reset_index()

last_date_df


,symbol,date,close,high,low,open,volume
0,AFYA,2024-03-08,20.71,20.92,20.074,20.22,131873.0
1,EDU,2024-03-08,90.17,91.87,87.090,90.00,2687256.0
2,GRND,2024-03-08,9.35,9.67,9.050,9.33,1352025.0
3,MTN,2024-03-08,223.76,225.79,218.150,221.90,720663.0
4,NMM,2024-03-08,44.30,44.33,41.400,41.50,513793.0
5,ONON,2024-03-08,33.09,33.18,32.385,32.62,6576098.0
6,PRSO,2024-03-08,1.83,1.85,1.610,1.63,597897.0


In [21]:
# re-index the dataframe
chart_historical_df.reset_index(drop=True, inplace=True)
# view head of your main df
print(f'Length of historical data: {len(chart_historical_df.index)}')

Length of historical data: 721


In [22]:
"""
YOUR DATAFRAMES TO MOVE FORWARD WITH:
"""
#### INFO / NEWS DATAFRAME ####
print(f'1 - NEWS & STOCK INFO (trending news etc.):\n{info_news_df}\n')

#### IEX (API) - HISTORICAL DATA FRAME ####
print('2a - IEX HISTORICAL data (START):')
print(f'{chart_historical_df.head()}\n')
print('2b - IEX HISTORICAL data (END):')
print(f'{chart_historical_df.tail()}\n')

1 - NEWS & STOCK INFO (trending news etc.):
  Symbol Exchange marketCapType
0    EDU     NYSE    Large caps
1   ONON     NYSE    Large caps
2    MTN     NYSE       Midlers
3   AFYA   NASDAQ    Small caps
4   GRND     NYSE    Small caps
5    NMM     NYSE    Small caps
6   PRSO   NASDAQ        Shrimp

2a - IEX HISTORICAL data (START):
  symbol        date   close    high       low    open     volume
0    EDU  2023-10-11   64.43   65.92   63.2001   63.36  1683723.0
1   ONON  2023-10-11   24.66   25.61   24.6500   25.50  4495999.0
2   AFYA  2023-10-11   15.59   15.71   15.2600   15.26   124125.0
3   GRND  2023-10-11    5.88    5.94    5.7800    5.89   178041.0
4    MTN  2023-10-11  215.87  218.11  213.1580  213.61   382468.0

2b - IEX HISTORICAL data (END):
    symbol        date  close   high     low   open     volume
716    EDU  2024-03-08  90.17  91.87  87.090  90.00  2687256.0
717   GRND  2024-03-08   9.35   9.67   9.050   9.33  1352025.0
718   ONON  2024-03-08  33.09  33.18  32.385  3

In [23]:
#### TRADINGVIEW (IMPORTED) - DATAFRAME (TODAY'S DATA) ####
print('3 - TRADINGVIEW (IMPORTED) - DATAFRAME (TODAY):')
print(f'U.S. stocks returned from the pre-market gap up screener = {len(tv_L2_df.index)}')
tv_L2_df

3 - TRADINGVIEW (IMPORTED) - DATAFRAME (TODAY):
U.S. stocks returned from the pre-market gap up screener = 7


,Symbol,Description,Exchange,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Sector,Relative Volume at Time,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 day,Relative Volume 1 week,Relative Volume 1 month,New high 1 month,New high 1 month - Currency,New high 3 months,New high 3 months - Currency,New high 6 months,New high 6 months - Currency,New high 52 weeks,New high 52 weeks - Currency,New high All Time,New high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower",marketCapType
0,EDU,"New Oriental Education & Technology Group, Inc...",NYSE,1.497948e+10,USD,90.17,USD,90.47,USD,0.13,USD,0.144172,0.332705,1.654587e+08,2687256,9137894,3692,1524569.7,1768768.69999998,1785831.22222223,5.488575,3.932455,3.286652,89.710000,49.21137368,Consumer Services,1.490119,1.1611879,0.24545807,0.5030385,2.555885,6.406386,7.290278,2.120533,0.976995,0.602067,0.689005,1.893552,1.07552366,0.29500058,98.200,USD,98.20,USD,98.200,USD,98.20,USD,199.7400,USD,90.250,USD,90.25,USD,90.25,USD,90.25,USD,90.25,USD,90.36,USD,91.87,USD,98.20,USD,98.200,USD,54007.85500667,USD,90.15500,90.143750,89.99200,90.014231,90.274081,89.98081,89.687539,90.243335,89.940500,89.637665,Large caps
1,ONON,On Holding AG,NYSE,1.047365e+10,USD,33.09,USD,33.84,USD,0.16,USD,0.483530,2.266546,1.876031e+08,6576098,30306742,34748,5341704.4,4837171.2,5054020.62222222,3.581905,3.184276,3.365261,32.885000,118.68723099,Consumer Non-Durables,0.979479,1.6530961,2.179955,2.179955,2.480428,3.250074,3.452546,2.385260,1.209855,0.724526,0.673585,1.319468,1.33320761,0.36159538,35.580,USD,35.58,USD,35.580,USD,37.08,USD,55.8700,USD,33.100,USD,33.11,USD,33.11,USD,33.11,USD,33.11,USD,33.11,USD,33.18,USD,34.72,USD,35.125,USD,752262.56024297,USD,33.07398,33.066862,33.01400,32.953000,33.110527,33.01462,32.918713,33.086952,32.934450,32.781948,Large caps
2,MTN,"Vail Resorts, Inc.",NYSE,8.495413e+09,USD,223.76,USD,224.60,USD,0.24,USD,0.107258,0.375402,3.752368e+07,720663,3615720,1194,535531.6,400322.9,393717.48888889,3.635113,3.531393,2.202390,222.566667,41.53472055,Consumer Services,1.212317,1.1425825,0.7754122,1.1195556,5.844120,10.808708,8.242927,4.580450,1.703201,1.009830,0.718746,1.464385,2.06868201,0.48192066,236.920,USD,236.92,USD,254.780,USD,258.13,USD,376.2400,USD,224.365,USD,224.91,USD,225.47,USD,225.47,USD,225.47,USD,225.79,USD,225.79,USD,235.81,USD,235.810,USD,50869.08450704,USD,224.14100,224.090000,224.43300,224.837692,225.108001,224.38350,223.658999,225.762724,224.692750,223.622776,Midlers
3,AFYA,Afya Limited,NASDAQ,1.941000e+09,USD,20.71,USD,21.00,USD,0.29,USD,1.400290,1.400290,1.798361e+07,131873,355394,992,66157.6,89576.43333333,155568.8,4.214407,2.344296,2.535

In [24]:
"""PICK UP HERE WHERE YOU LEFT OFF"""
# you need to get the chart dataframe to a place where it has all the data you need in there. 
# it would be more next level if we let it rip into live stock feeds from here. all setup with the correct lines.
# in the charts live feed we could use the tradingview html widget to display the charts.


'PICK UP HERE WHERE YOU LEFT OFF'

| KEY LEVELS (Support & Resistance) |
|-|
| TECHNICAL INDICATOR CALCULATIONS FROM HISTORICAL DATA |

in order of priority of in scope. 
thinking of moving some of the levels out as i test them and see what they look like.
for instance i am thinking of eventually completely replacing MAs with EMAs

In [25]:
"calculate key levels from historical data"
# save historical data to a csv
chart_historical_df.to_csv(f'chart_historical_gpt_data{today_date_str}.csv', index=False)

chart_historical_df.head()

,symbol,date,close,high,low,open,volume
0,EDU,2023-10-11,64.43,65.92,63.2001,63.36,1683723.0
1,ONON,2023-10-11,24.66,25.61,24.6500,25.50,4495999.0
2,AFYA,2023-10-11,15.59,15.71,15.2600,15.26,124125.0
3,GRND,2023-10-11,5.88,5.94,5.7800,5.89,178041.0
4,MTN,2023-10-11,215.87,218.11,213.1580,213.61,382468.0


In [26]:
"""
IMPORTANT:
for effective plotting and time series analysis, it's crucial that the date column in the df is in the datetime format rather than an object (string). 
"""
chart_historical_df['date'] = pd.to_datetime(chart_historical_df['date'])
chart_historical_df.set_index('date', inplace=True)

# view the multi-indexed dataframe
chart_historical_df.head()

,symbol,close,high,low,open,volume
date,,,,,,
2023-10-11,EDU,64.43,65.92,63.2001,63.36,1683723.0
2023-10-11,ONON,24.66,25.61,24.6500,25.50,4495999.0
2023-10-11,AFYA,15.59,15.71,15.2600,15.26,124125.0
2023-10-11,GRND,5.88,5.94,5.7800,5.89,178041.0
2023-10-11,MTN,215.87,218.11,213.1580,213.61,382468.0


In [27]:
"""SIMPLE MOVING AVERAGES (SMA)"""
def calculate_sma(dataframe, period):
    return dataframe['close'].rolling(window=period).mean()

chart_historical_df['SMA5'] = calculate_sma(chart_historical_df, 5)
chart_historical_df['SMA10'] = calculate_sma(chart_historical_df, 10)
chart_historical_df['SMA15'] = calculate_sma(chart_historical_df, 15)

chart_historical_df.tail(10)


,symbol,close,high,low,open,volume,SMA5,SMA10,SMA15
date,,,,,,,,,
2024-03-07,EDU,90.47,93.9105,89.760,93.110,2243620.0,81.930,73.586,64.578000
2024-03-07,GRND,8.71,8.7800,8.240,8.350,1338915.0,38.456,70.819,58.840667
2024-03-07,PRSO,1.64,1.7200,1.575,1.575,132904.0,34.746,48.112,58.406000
2024-03-08,MTN,223.76,225.7900,218.150,221.900,720663.0,73.094,67.224,71.111333
2024-03-08,NMM,44.30,44.3300,41.400,41.500,513793.0,73.776,69.631,73.960000
2024-03-08,EDU,90.17,91.8700,87.090,90.000,2687256.0,73.716,77.823,73.629333
2024-03-08,GRND,9.35,9.6700,9.050,9.330,1352025.0,73.844,56.150,71.827333
2024-03-08,ONON,33.09,33.1800,32.385,32.620,6576098.0,80.134,57.440,58.786000
2024-03-08,AFYA,20.71,20.9200,20.074,20.220,131873.0,39.524,56.309,57.990667


In [28]:
"""EXPONENTIAL MOVING AVERAGES (EMA)"""
def calculate_ema(dataframe, period):
    return dataframe['close'].ewm(span=period, adjust=False).mean()

# calculate EMAs for the specified periods
chart_historical_df['EMA5'] = calculate_ema(chart_historical_df, 5)
chart_historical_df['EMA10'] = calculate_ema(chart_historical_df, 10)
chart_historical_df['EMA20'] = calculate_ema(chart_historical_df, 20)
chart_historical_df['EMA50'] = calculate_ema(chart_historical_df, 50)

chart_historical_df.tail(10)


,symbol,close,high,low,open,volume,SMA5,SMA10,SMA15,EMA5,EMA10,EMA20,EMA50
date,,,,,,,,,,,,,
2024-03-07,EDU,90.47,93.9105,89.760,93.110,2243620.0,81.930,73.586,64.578000,66.726058,67.131172,66.411412,63.845795
2024-03-07,GRND,8.71,8.7800,8.240,8.350,1338915.0,38.456,70.819,58.840667,47.387372,56.509141,60.916039,61.683607
2024-03-07,PRSO,1.64,1.7200,1.575,1.575,132904.0,34.746,48.112,58.406000,32.138248,46.532933,55.270702,59.328956
2024-03-08,MTN,223.76,225.7900,218.150,221.900,720663.0,73.094,67.224,71.111333,96.012165,78.756036,71.317302,65.777232
2024-03-08,NMM,44.30,44.3300,41.400,41.500,513793.0,73.776,69.631,73.960000,78.774777,72.491302,68.744226,64.934988
2024-03-08,EDU,90.17,91.8700,87.090,90.000,2687256.0,73.716,77.823,73.629333,82.573185,75.705611,70.784776,65.924596
2024-03-08,GRND,9.35,9.6700,9.050,9.330,1352025.0,73.844,56.150,71.827333,58.165456,63.640955,64.933845,63.705985
2024-03-08,ONON,33.09,33.1800,32.385,32.620,6576098.0,80.134,57.440,58.786000,49.806971,58.086236,61.901097,62.505358
2024-03-08,AFYA,20.71,20.9200,20.074,20.220,131873.0,39.524,56.309,57.990667,40.107981,51.290556,57.978136,60.866324


In [29]:
"""MOVING AVERAGE CONVERGENCE DIVERGENCE (MACD)"""
# calculate the MACD line and the Signal line
chart_historical_df['MACD'] = calculate_ema(chart_historical_df, 12) - calculate_ema(chart_historical_df, 26)
chart_historical_df['MACD_Signal'] = chart_historical_df['MACD'].ewm(span=9, adjust=False).mean()

# MACD Histogram -> useful for visualizing momentum and potential buy/sell signals
chart_historical_df['MACD_Histogram'] = chart_historical_df['MACD'] - chart_historical_df['MACD_Signal']

chart_historical_df.tail(10)

,symbol,close,high,low,open,volume,SMA5,SMA10,SMA15,EMA5,EMA10,EMA20,EMA50,MACD,MACD_Signal,MACD_Histogram
date,,,,,,,,,,,,,,,,
2024-03-07,EDU,90.47,93.9105,89.760,93.110,2243620.0,81.930,73.586,64.578000,66.726058,67.131172,66.411412,63.845795,1.334994,2.023025,-0.688030
2024-03-07,GRND,8.71,8.7800,8.240,8.350,1338915.0,38.456,70.819,58.840667,47.387372,56.509141,60.916039,61.683607,-3.422870,0.933846,-4.356716
2024-03-07,PRSO,1.64,1.7200,1.575,1.575,132904.0,34.746,48.112,58.406000,32.138248,46.532933,55.270702,59.328956,-7.675524,-0.788028,-6.887496
2024-03-08,MTN,223.76,225.7900,218.150,221.900,720663.0,73.094,67.224,71.111333,96.012165,78.756036,71.317302,65.777232,6.799070,0.729391,6.069678
2024-03-08,NMM,44.30,44.3300,41.400,41.500,513793.0,73.776,69.631,73.960000,78.774777,72.491302,68.744226,64.934988,3.746184,1.332750,2.413434
2024-03-08,EDU,90.17,91.8700,87.090,90.000,2687256.0,73.716,77.823,73.629333,82.573185,75.705611,70.784776,65.924596,4.970775,2.060355,2.910420
2024-03-08,GRND,9.35,9.6700,9.050,9.330,1352025.0,73.844,56.150,71.827333,58.165456,63.640955,64.933845,63.705985,-0.573614,1.533561,-2.107175
2024-03-08,ONON,33.09,33.1800,32.385,32.620,6576098.0,80.134,57.440,58.786000,49.806971,58.086236,61.901097,62.505358,-3.017182,0.623413,-3.640594
2024-03-08,AFYA,20.71,20.9200,20.074,20.220,131873.0,39.524,56.309,57.990667,40.107981,51.290556,57.978136,60.866324,-5.884853,-0.678240,-5.206612


In [30]:
#### DEV - DATA ORGANIZATION ####
chart_historical_df.columns

Index(['symbol', 'close', 'high', 'low', 'open', 'volume', 'SMA5', 'SMA10',
       'SMA15', 'EMA5', 'EMA10', 'EMA20', 'EMA50', 'MACD', 'MACD_Signal',
       'MACD_Histogram'],
      dtype='object')

In [31]:
"""BOLLINGER BANDS (BB)"""
# BB 1 of 2 -> standard bollinger bands (20D CLOSE $)
period = 20 # SMA periods
std_dev_multiplier = 2 # standard deviation multiplier

# line 1: middle band -> simple moving average (SMA)
chart_historical_df[f'middle_band_SMA{period}'] = chart_historical_df['close'].rolling(window=period).mean()

# not a line: standard deviation -> used to calculate the upper and lower bands
chart_historical_df[f'std_dev_SMA{period}'] = chart_historical_df['close'].rolling(window=period).std()

# lines 2 & 3: upper and lower bands
chart_historical_df[f'upper_band_SMA{period}'] = chart_historical_df[f'middle_band_SMA{period}'] + (chart_historical_df[f'std_dev_SMA{period}'] * std_dev_multiplier)
chart_historical_df[f'lower_band_SMA{period}'] = chart_historical_df[f'middle_band_SMA{period}'] - (chart_historical_df[f'std_dev_SMA{period}'] * std_dev_multiplier)


# BB 2 of 2 -> typical price bollinger bands (20D TYPICAL $)
chart_historical_df['typical_price'] = (chart_historical_df['high'] + chart_historical_df['low'] + chart_historical_df['close']) / 3

# Calculate the SMA of the Typical Price and name it 'middle_band_typical_SMA20'
chart_historical_df[f'middle_band_typical_SMA{period}'] = chart_historical_df['typical_price'].rolling(window=period).mean()

# Calculate the standard deviation of the Typical Price
chart_historical_df[f'std_dev_typical_SMA{period}'] = chart_historical_df['typical_price'].rolling(window=period).std()

# Calculate the upper and lower bands for the Typical Price and name them accordingly
chart_historical_df[f'upper_band_typical_SMA{period}'] = chart_historical_df[f'middle_band_typical_SMA{period}'] + (chart_historical_df[f'std_dev_typical_SMA{period}'] * std_dev_multiplier)
chart_historical_df[f'lower_band_typical_SMA{period}'] = chart_historical_df[f'middle_band_typical_SMA{period}'] - (chart_historical_df[f'std_dev_typical_SMA{period}'] * std_dev_multiplier)

# view df with new (2) BBs (6 new columns)
# chart_historical_df.iloc[-1] # last row slice to check the new columns
chart_historical_df.tail(2)


,symbol,close,high,low,open,volume,SMA5,SMA10,SMA15,EMA5,EMA10,EMA20,EMA50,MACD,MACD_Signal,MACD_Histogram,middle_band_SMA20,std_dev_SMA20,upper_band_SMA20,lower_band_SMA20,typical_price,middle_band_typical_SMA20,std_dev_typical_SMA20,upper_band_typical_SMA20,lower_band_typical_SMA20
date,,,,,,,,,,,,,,,,,,,,,,,,,
2024-03-08,AFYA,20.71,20.92,20.074,20.22,131873.0,39.524,56.309,57.990667,40.107981,51.290556,57.978136,60.866324,-5.884853,-0.67824,-5.206612,63.2145,75.588413,214.391326,-87.962326,20.568000,63.169658,75.865760,214.901179,-88.561862
2024-03-08,PRSO,1.83,1.85,1.610,1.63,597897.0,31.030,52.403,56.764000,27.348654,42.297728,52.630694,58.551174,-9.570637,-2.45672,-7.113917,63.2275,75.577275,214.382050,-87.927050,1.763333,63.178992,75.857795,214.894582,-88.536598


In [32]:
chart_historical_df.head(5)

,symbol,close,high,low,open,volume,SMA5,SMA10,SMA15,EMA5,EMA10,EMA20,EMA50,MACD,MACD_Signal,MACD_Histogram,middle_band_SMA20,std_dev_SMA20,upper_band_SMA20,lower_band_SMA20,typical_price,middle_band_typical_SMA20,std_dev_typical_SMA20,upper_band_typical_SMA20,lower_band_typical_SMA20
date,,,,,,,,,,,,,,,,,,,,,,,,,
2023-10-11,EDU,64.43,65.92,63.2001,63.36,1683723.0,NaN,NaN,NaN,64.430000,64.430000,64.430000,64.430000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,64.516700,NaN,NaN,NaN,NaN
2023-10-11,ONON,24.66,25.61,24.6500,25.50,4495999.0,NaN,NaN,NaN,51.173333,57.199091,60.642381,62.870392,-3.172536,-0.634507,-2.538028,NaN,NaN,NaN,NaN,24.973333,NaN,NaN,NaN,NaN
2023-10-11,AFYA,15.59,15.71,15.2600,15.26,124125.0,NaN,NaN,NaN,39.312222,49.633802,56.351678,61.016259,-6.345519,-1.776709,-4.568809,NaN,NaN,NaN,NaN,15.520000,NaN,NaN,NaN,NaN
2023-10-11,GRND,5.88,5.94,5.7800,5.89,178041.0,NaN,NaN,NaN,28.168148,41.678565,51.544852,58.854053,-9.533748,-3.328117,-6.205631,NaN,NaN,NaN,NaN,5.866667,NaN,NaN,NaN,NaN
2023-10-11,MTN,215.87,218.11,213.1580,213.61,382468.0,65.286,NaN,NaN,90.735432,73.349735,67.194866,65.011541,4.828338,-1.696826,6.525164,NaN,NaN,NaN,NaN,215.712667,NaN,NaN,NaN,NaN


In [33]:
# drop rows that contain NAs (pulled extra data earlier for this purpose)
chart_historical_df = chart_historical_df.dropna()

# view the df
chart_historical_df.head(5)

,symbol,close,high,low,open,volume,SMA5,SMA10,SMA15,EMA5,EMA10,EMA20,EMA50,MACD,MACD_Signal,MACD_Histogram,middle_band_SMA20,std_dev_SMA20,upper_band_SMA20,lower_band_SMA20,typical_price,middle_band_typical_SMA20,std_dev_typical_SMA20,upper_band_typical_SMA20,lower_band_typical_SMA20
date,,,,,,,,,,,,,,,,,,,,,,,,,
2023-10-13,GRND,5.88,5.930,5.84,5.87,250162.0,26.032,44.3224,34.546933,20.323658,30.507861,40.136384,51.022761,-10.414263,-6.192043,-4.222220,42.2317,61.851982,165.935665,-81.472265,5.883333,42.316842,61.880542,166.077926,-81.444243
2023-10-13,MTN,215.08,215.340,211.64,211.64,323891.0,56.574,63.5434,47.352267,85.242439,64.066432,56.797681,57.456378,4.872198,-3.979195,8.851393,49.7642,72.886605,195.537410,-96.009010,214.020000,49.792007,72.774492,195.340991,-95.756978
2023-10-16,ONON,24.89,25.340,23.81,23.81,4008727.0,56.818,44.6984,48.552933,65.124959,56.943444,53.758854,56.179265,1.621395,-2.859077,4.480472,49.7757,72.882453,195.540607,-95.989207,24.680000,49.777340,72.779787,195.336913,-95.782233
2023-10-16,AFYA,15.83,16.160,15.36,15.44,292307.0,55.416,40.0164,48.022933,48.693306,49.468273,50.146582,54.596941,-1.666743,-2.620610,0.953867,49.7877,72.876548,195.540796,-95.965396,15.783333,49.790507,72.773287,195.337080,-95.756066
2023-10-16,EDU,62.82,63.365,61.23,61.42,622083.0,64.900,45.5784,51.182267,53.402204,51.895859,51.353574,54.919414,-0.475435,-2.191575,1.716140,52.6347,72.179849,196.994397,-91.724997,62.471667,52.620757,72.072469,196.765695,-91.524181


| CHARTS |
|-|

In [34]:
# date column check data type
chart_historical_df.index

DatetimeIndex(['2023-10-13', '2023-10-13', '2023-10-16', '2023-10-16',
               '2023-10-16', '2023-10-16', '2023-10-16', '2023-10-16',
               '2023-10-16', '2023-10-17',
               ...
               '2024-03-07', '2024-03-07', '2024-03-07', '2024-03-08',
               '2024-03-08', '2024-03-08', '2024-03-08', '2024-03-08',
               '2024-03-08', '2024-03-08'],
              dtype='datetime64[ns]', name='date', length=702, freq=None)

In [35]:
# IMPORTANT: reset index before saving to Excel (need this shit flat so we don't lose the 'date' index)
chart_historical_df.reset_index(inplace=True)

In [36]:
chart_historical_df.index

RangeIndex(start=0, stop=702, step=1)

In [37]:
chart_historical_df.head(5)

,date,symbol,close,high,low,open,volume,SMA5,SMA10,SMA15,EMA5,EMA10,EMA20,EMA50,MACD,MACD_Signal,MACD_Histogram,middle_band_SMA20,std_dev_SMA20,upper_band_SMA20,lower_band_SMA20,typical_price,middle_band_typical_SMA20,std_dev_typical_SMA20,upper_band_typical_SMA20,lower_band_typical_SMA20
0,2023-10-13,GRND,5.88,5.930,5.84,5.87,250162.0,26.032,44.3224,34.546933,20.323658,30.507861,40.136384,51.022761,-10.414263,-6.192043,-4.222220,42.2317,61.851982,165.935665,-81.472265,5.883333,42.316842,61.880542,166.077926,-81.444243
1,2023-10-13,MTN,215.08,215.340,211.64,211.64,323891.0,56.574,63.5434,47.352267,85.242439,64.066432,56.797681,57.456378,4.872198,-3.979195,8.851393,49.7642,72.886605,195.537410,-96.009010,214.020000,49.792007,72.774492,195.340991,-95.756978
2,2023-10-16,ONON,24.89,25.340,23.81,23.81,4008727.0,56.818,44.6984,48.552933,65.124959,56.943444,53.758854,56.179265,1.621395,-2.859077,4.480472,49.7757,72.882453,195.540607,-95.989207,24.680000,49.777340,72.779787,195.336913,-95.782233
3,2023-10-16,AFYA,15.83,16.160,15.36,15.44,292307.0,55.416,40.0164,48.022933,48.693306,49.468273,50.146582,54.596941,-1.666743,-2.620610,0.953867,49.7877,72.876548,195.540796,-95.965396,15.783333,49.790507,72.773287,195.337080,-95.756066
4,2023-10-16,EDU,62.82,63.365,61.23,61.42,622083.0,64.900,45.5784,51.182267,53.402204,51.895859,51.353574,54.919414,-0.475435,-2.191575,1.716140,52.6347,72.179849,196.994397,-91.724997,62.471667,52.620757,72.072469,196.765695,-91.524181


| CHARTS - xlwings |
|-|

In [38]:
# save name for todays returend gap up stocks historical data
chart_data_file = f'chart_data_{today_date_str}.xlsx'

# Save the DataFrame to an Excel file
chart_historical_df.to_excel(chart_data_file, index=False)

# Use xlwings to open the file in Excel
app = xw.App(visible=True)
book = app.books.open(chart_data_file)

In [39]:
# create a dataframe

In [40]:
tv_L2_df

,Symbol,Description,Exchange,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Sector,Relative Volume at Time,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 day,Relative Volume 1 week,Relative Volume 1 month,New high 1 month,New high 1 month - Currency,New high 3 months,New high 3 months - Currency,New high 6 months,New high 6 months - Currency,New high 52 weeks,New high 52 weeks - Currency,New high All Time,New high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower",marketCapType
0,EDU,"New Oriental Education & Technology Group, Inc...",NYSE,1.497948e+10,USD,90.17,USD,90.47,USD,0.13,USD,0.144172,0.332705,1.654587e+08,2687256,9137894,3692,1524569.7,1768768.69999998,1785831.22222223,5.488575,3.932455,3.286652,89.710000,49.21137368,Consumer Services,1.490119,1.1611879,0.24545807,0.5030385,2.555885,6.406386,7.290278,2.120533,0.976995,0.602067,0.689005,1.893552,1.07552366,0.29500058,98.200,USD,98.20,USD,98.200,USD,98.20,USD,199.7400,USD,90.250,USD,90.25,USD,90.25,USD,90.25,USD,90.25,USD,90.36,USD,91.87,USD,98.20,USD,98.200,USD,54007.85500667,USD,90.15500,90.143750,89.99200,90.014231,90.274081,89.98081,89.687539,90.243335,89.940500,89.637665,Large caps
1,ONON,On Holding AG,NYSE,1.047365e+10,USD,33.09,USD,33.84,USD,0.16,USD,0.483530,2.266546,1.876031e+08,6576098,30306742,34748,5341704.4,4837171.2,5054020.62222222,3.581905,3.184276,3.365261,32.885000,118.68723099,Consumer Non-Durables,0.979479,1.6530961,2.179955,2.179955,2.480428,3.250074,3.452546,2.385260,1.209855,0.724526,0.673585,1.319468,1.33320761,0.36159538,35.580,USD,35.58,USD,35.580,USD,37.08,USD,55.8700,USD,33.100,USD,33.11,USD,33.11,USD,33.11,USD,33.11,USD,33.11,USD,33.18,USD,34.72,USD,35.125,USD,752262.56024297,USD,33.07398,33.066862,33.01400,32.953000,33.110527,33.01462,32.918713,33.086952,32.934450,32.781948,Large caps
2,MTN,"Vail Resorts, Inc.",NYSE,8.495413e+09,USD,223.76,USD,224.60,USD,0.24,USD,0.107258,0.375402,3.752368e+07,720663,3615720,1194,535531.6,400322.9,393717.48888889,3.635113,3.531393,2.202390,222.566667,41.53472055,Consumer Services,1.212317,1.1425825,0.7754122,1.1195556,5.844120,10.808708,8.242927,4.580450,1.703201,1.009830,0.718746,1.464385,2.06868201,0.48192066,236.920,USD,236.92,USD,254.780,USD,258.13,USD,376.2400,USD,224.365,USD,224.91,USD,225.47,USD,225.47,USD,225.47,USD,225.79,USD,225.79,USD,235.81,USD,235.810,USD,50869.08450704,USD,224.14100,224.090000,224.43300,224.837692,225.108001,224.38350,223.658999,225.762724,224.692750,223.622776,Midlers
3,AFYA,Afya Limited,NASDAQ,1.941000e+09,USD,20.71,USD,21.00,USD,0.29,USD,1.400290,1.400290,1.798361e+07,131873,355394,992,66157.6,89576.43333333,155568.8,4.214407,2.344296,2.535

In [41]:
# tview_iex_chart = Chart()
# chart.set(chart_)


In [42]:
# # TradingView df
# tv_L2_df.head(2)


In [43]:
# TradingView df cols
tv_L2_df.columns

Index(['Symbol', 'Description', 'Exchange', 'Market capitalization',
       'Market capitalization - Currency', 'Price', 'Price - Currency',
       'Pre-market Open', 'Pre-market Open - Currency', 'Pre-market Change',
       'Pre-market Change - Currency', 'Pre-market Change %',
       'Pre-market Gap %', 'Float shares outstanding', 'Volume 1 day',
       'Volume 1 week', 'Pre-market Volume', 'Average Volume 10 days',
       'Average Volume 30 days', 'Average Volume 90 days', 'Volatility 1 day',
       'Volatility 1 week', 'Volatility 1 month',
       'Volume Weighted Average Price 1 day', 'Price to earnings ratio',
       'Sector', 'Relative Volume at Time', 'Beta 1 year', 'Beta 3 years',
       'Beta 5 years', 'Relative Volume 1 minute', 'Relative Volume 5 minutes',
       'Relative Volume 15 minutes', 'Relative Volume 30 minutes',
       'Relative Volume 1 hour', 'Relative Volume 2 hours',
       'Relative Volume 4 hours', 'Relative Volume 1 day',
       'Relative Volume 1 week', 'R

| CALCULATIONS - J. Welles Wilder Jr (technical indicators) |
|-|
| *displayed underneath the chart not overlayed* |
| START |

In [44]:
"""RELATIVE STRENGTH INDEX (RSI)"""
def calculate_rsi(dataframe, periods=14):
    close_delta = dataframe['close'].diff()
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)

    ma_up = up.rolling(window=periods).mean()
    ma_down = down.rolling(window=periods).mean()

    rsi = ma_up / (ma_up + ma_down) * 100
    return rsi

# Adding the RSI to your DataFrame
chart_historical_df['RSI'] = calculate_rsi(chart_historical_df)

chart_historical_df.tail(10)


,date,symbol,close,high,low,open,volume,SMA5,SMA10,SMA15,EMA5,EMA10,EMA20,EMA50,MACD,MACD_Signal,MACD_Histogram,middle_band_SMA20,std_dev_SMA20,upper_band_SMA20,lower_band_SMA20,typical_price,middle_band_typical_SMA20,std_dev_typical_SMA20,upper_band_typical_SMA20,lower_band_typical_SMA20,RSI
692,2024-03-07,EDU,90.47,93.9105,89.760,93.110,2243620.0,81.930,73.586,64.578000,66.726058,67.131172,66.411412,63.845795,1.334994,2.023025,-0.688030,63.8345,74.668071,213.170642,-85.501642,91.380167,63.959698,75.024510,214.008719,-86.089322,49.821182
693,2024-03-07,GRND,8.71,8.7800,8.240,8.350,1338915.0,38.456,70.819,58.840667,47.387372,56.509141,60.916039,61.683607,-3.422870,0.933846,-4.356716,62.5685,75.410959,213.390419,-88.253419,8.576667,62.677032,75.775078,214.227188,-88.873124,50.022965
694,2024-03-07,PRSO,1.64,1.7200,1.575,1.575,132904.0,34.746,48.112,58.406000,32.138248,46.532933,55.270702,59.328956,-7.675524,-0.788028,-6.887496,62.5700,75.409683,213.389367,-88.249367,1.645000,62.676448,75.775572,214.227593,-88.874697,48.663038
695,2024-03-08,MTN,223.76,225.7900,218.150,221.900,720663.0,73.094,67.224,71.111333,96.012165,78.756036,71.317302,65.777232,6.799070,0.729391,6.069678,72.1065,83.141127,238.388755,-94.175755,222.566667,72.145282,83.350580,238.846442,-94.555879,58.108828
696,2024-03-08,NMM,44.30,44.3300,41.400,41.500,513793.0,73.776,69.631,73.960000,78.774777,72.491302,68.744226,64.934988,3.746184,1.332750,2.413434,73.3070,82.524613,238.356225,-91.742225,43.343333,73.297915,82.753008,238.803930,-92.208100,48.254404
697,2024-03-08,EDU,90.17,91.8700,87.090,90.000,2687256.0,73.716,77.823,73.629333,82.573185,75.705611,70.784776,65.924596,4.970775,2.060355,2.910420,66.8625,75.255954,217.374408,-83.649408,89.710000,66.838248,75.514731,217.867711,-84.191214,51.863735
698,2024-03-08,GRND,9.35,9.6700,9.050,9.330,1352025.0,73.844,56.150,71.827333,58.165456,63.640955,64.933845,63.705985,-0.573614,1.533561,-2.107175,62.5915,76.008914,214.609328,-89.426328,9.356667,62.578840,76.268447,215.115734,-89.958054,41.763045
699,2024-03-08,ONON,33.09,33.1800,32.385,32.620,6576098.0,80.134,57.440,58.786000,49.806971,58.086236,61.901097,62.505358,-3.017182,0.623413,-3.640594,63.8380,75.270117,214.378234,-86.702234,32.885000,63.810423,75.540429,214.891281,-87.270435,50.019409
700,2024-03-08,AFYA,20.71,20.9200,20.074,20.220,131873.0,39.524,56.309,57.990667,40.107981,51.290556,57.978136,60.866324,-5.884853,-0.678240,-5.206612,63.2145,75.588413,214.391326,-87.962326,20.568000,63.169658,75.865760,214.901179,-88.561862,50.020704
701,2024-03-08,PRSO,1.83,1.8500,1.610,1.630,597897.0,31.030,52.403,56.764000,27.348654,42.297728,52.630694,58.551174,-9.570637,-2.456720,-7.113917,63.2275,75.577275,214.382050,-87.927050,1.763333,63.178992,75.857795,214.894582,-88.536598,49.724723


In [45]:
"""AVERAGE TRUE RANGE (ATR)"""
def calculate_atr(dataframe, period=14):
    high_low = dataframe['high'] - dataframe['low']
    high_close = (dataframe['high'] - dataframe['close'].shift()).abs()
    low_close = (dataframe['low'] - dataframe['close'].shift()).abs()

    # Calculate the True Range (TR) as the maximum of the three measures
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)

    # Calculate the ATR by taking the rolling mean of the TR
    atr = tr.rolling(window=period).mean()
    
    return atr

# Adding the ATR to your DataFrame
chart_historical_df['ATR'] = calculate_atr(chart_historical_df)

chart_historical_df.tail(10)

,date,symbol,close,high,low,open,volume,SMA5,SMA10,SMA15,EMA5,EMA10,EMA20,EMA50,MACD,MACD_Signal,MACD_Histogram,middle_band_SMA20,std_dev_SMA20,upper_band_SMA20,lower_band_SMA20,typical_price,middle_band_typical_SMA20,std_dev_typical_SMA20,upper_band_typical_SMA20,lower_band_typical_SMA20,RSI,ATR
692,2024-03-07,EDU,90.47,93.9105,89.760,93.110,2243620.0,81.930,73.586,64.578000,66.726058,67.131172,66.411412,63.845795,1.334994,2.023025,-0.688030,63.8345,74.668071,213.170642,-85.501642,91.380167,63.959698,75.024510,214.008719,-86.089322,49.821182,87.469314
693,2024-03-07,GRND,8.71,8.7800,8.240,8.350,1338915.0,38.456,70.819,58.840667,47.387372,56.509141,60.916039,61.683607,-3.422870,0.933846,-4.356716,62.5685,75.410959,213.390419,-88.253419,8.576667,62.677032,75.775078,214.227188,-88.873124,50.022965,87.152171
694,2024-03-07,PRSO,1.64,1.7200,1.575,1.575,132904.0,34.746,48.112,58.406000,32.138248,46.532933,55.270702,59.328956,-7.675524,-0.788028,-6.887496,62.5700,75.409683,213.389367,-88.249367,1.645000,62.676448,75.775572,214.227593,-88.874697,48.663038,85.823250
695,2024-03-08,MTN,223.76,225.7900,218.150,221.900,720663.0,73.094,67.224,71.111333,96.012165,78.756036,71.317302,65.777232,6.799070,0.729391,6.069678,72.1065,83.141127,238.388755,-94.175755,222.566667,72.145282,83.350580,238.846442,-94.555879,58.108828,99.571821
696,2024-03-08,NMM,44.30,44.3300,41.400,41.500,513793.0,73.776,69.631,73.960000,78.774777,72.491302,68.744226,64.934988,3.746184,1.332750,2.413434,73.3070,82.524613,238.356225,-91.742225,43.343333,73.297915,82.753008,238.803930,-92.208100,48.254404,105.759321
697,2024-03-08,EDU,90.17,91.8700,87.090,90.000,2687256.0,73.716,77.823,73.629333,82.573185,75.705611,70.784776,65.924596,4.970775,2.060355,2.910420,66.8625,75.255954,217.374408,-83.649408,89.710000,66.838248,75.514731,217.867711,-84.191214,51.863735,104.706464
698,2024-03-08,GRND,9.35,9.6700,9.050,9.330,1352025.0,73.844,56.150,71.827333,58.165456,63.640955,64.933845,63.705985,-0.573614,1.533561,-2.107175,62.5915,76.008914,214.609328,-89.426328,9.356667,62.578840,76.268447,215.115734,-89.958054,41.763045,96.255750
699,2024-03-08,ONON,33.09,33.1800,32.385,32.620,6576098.0,80.134,57.440,58.786000,49.806971,58.086236,61.901097,62.505358,-3.017182,0.623413,-3.640594,63.8380,75.270117,214.378234,-86.702234,32.885000,63.810423,75.540429,214.891281,-87.270435,50.019409,83.912179
700,2024-03-08,AFYA,20.71,20.9200,20.074,20.220,131873.0,39.524,56.309,57.990667,40.107981,51.290556,57.978136,60.866324,-5.884853,-0.678240,-5.206612,63.2145,75.588413,214.391326,-87.962326,20.568000,63.169658,75.865760,214.901179,-88.561862,50.020704,83.954036
701,2024-03-08,PRSO,1.83,1.8500,1.610,1.630,597897.0,31.030,52.403,56.764000,27.348654,42.297728,52.630694,58.551174,-9.570637,-2.456720,-7.113917,63.2275,75.577275,214.382050,-87.927050,1.763333,63.178992,75.857795,214.894582,-88.536598,49.724723,84.442607


In [46]:
# START HERE
# see about if i should be doing a multi-index and group by if it makes the code more efficient and accurate and all that
# simple data plot soon to see what we are working with and if there are bugs

In [47]:
"""Average Directional Index (ADX)"""

'Average Directional Index (ADX)'

In [48]:
"""PARABOLIC SAR (PSAR)"""

'PARABOLIC SAR (PSAR)'

| J. Welles Wilder Jr (technical indicators) |
|-|
| END |

In [49]:
# LINE LEVELS not full calculation dataframe
# part 1 was calculations append
# part 2 is data points append
# part 3 extra - could be plotting past signals (might integrate ai in here like tensorflow)

| CHARTS |
|-|

# DELL FOCUS - Pre-market

In [50]:
# # $DELL
# dell_df = key_levels_df[key_levels_df['symbol'] == 'DELL'].copy()
# dell_df['date'] = pd.to_datetime(dell_df['date'])
# dell_df.set_index('date', inplace=True)
# dell_df.head(5)

In [51]:
# import mplfinance as mpf
# # candlestick chart
# # MANDATORY DATA = 'open', 'high', 'low', 'close', 'volume'

# """
# Part 1:    
# """
# # Part 1a: SMAs -> calculate
# dell_df['SMA20'] = dell_df['close'].rolling(window=20, min_periods=1).mean()
# dell_df['SMA50'] = dell_df['close'].rolling(window=50, min_periods=1).mean()
# # dell_df['SMA200'] = dell_df['close'].rolling(window=200, min_periods=1).mean()

# # Part 1b: VWAP -> calculate
# dell_df['VWAP'] = (dell_df['volume'] * (dell_df['high'] + dell_df['low'] + dell_df['close']) / 3).cumsum() / dell_df['volume'].cumsum()

# """   
# Part 2: 
# """
# add2plot = [
#     #### SMAs ####
#     mpf.make_addplot(dell_df['SMA20'], color='#7BAFD4', width=1.0, label='SMA 20'), # UNC blue
#     mpf.make_addplot(dell_df['SMA50'], color='#003087', width=1.0, label='SMA 50'), # DUKE blue
#     # mpf.make_addplot(dell_df['SMA200'], color='#041E42', width=1.0, label='SMA 200'), # HOYAS blue
#     #### VWAP ####
#     mpf.make_addplot(dell_df['VWAP'], color='#B31B1B', width=1.0, label='VWAP') # BIG RED
# ]

# """   
# Part 3: styling not iterative changes related to data (i.e., color, font, etc.)
# """
# # Use a dark theme style and set text color to white
# mpf_style = mpf.make_mpf_style(base_mpf_style='nightclouds', rc={'text.color': 'white'})

# # Plot the candlestick chart with the moving averages
# fig, axes = mpf.plot(dell_df,
#                      type='candle',
#                      addplot=add2plot,
#                      volume=True,
#                     #  style=mpf_style,
#                      style=s, 
#                      # WTF look at docs (it's not erroring out but going to default style just figure out what that is i think)
#                      figratio=(15, 8),
#                      title='$DELL - Historical Data (6 Months)',
#                      tight_layout=True,
#                      returnfig=True)

# # Add a watermark if needed
# fig.text(0.5, 0.5, 'DELL', fontsize=50, color='gray', alpha=0.5, ha='center', va='center', rotation=0, transform=fig.transFigure)

# plt.show()


In [52]:
# dell_df.head(5)

In [53]:
""" 
ISSUE: some weird shit with the style param
nbd tho ctfo. 

workaround =  is first successfully run with style=mpf_style, then swap it for style=s (dark mode - but common view like Ortex)  
the blue shit and mpf_style is actually cool but might confuse user (i.e., blue = red and white = green) with the mpf_style
experienced similar shit like this years ago, idunno will get to it.

"""

' \nISSUE: some weird shit with the style param\nnbd tho ctfo. \n\nworkaround =  is first successfully run with style=mpf_style, then swap it for style=s (dark mode - but common view like Ortex)  \nthe blue shit and mpf_style is actually cool but might confuse user (i.e., blue = red and white = green) with the mpf_style\nexperienced similar shit like this years ago, idunno will get to it.\n\n'

In [54]:
# you're going to need different charts, one for the pre-market, and at least another for market open
# I'd like market opens to be a live feed but really we might already be in trading view watching the pine script execute

# Trend lines

| END OF IN PROGRESS WORK - NOTES BELOW |
|--|

In [55]:
# but first start with getting lines on that chart with STRV
# let's get those key levels with an output on them and figure out your strategy
# figure out when fibonacci extensions get activated
# i see one view with a grid layout of the stock charts above or below them the summary with the key levels, current price and all that
# i also see a view where when looking at individiual stocks we got data all over that shit. idgaf if it's "confusing for the user"

# you have your key_levels_df
# so start next from there okay

#### </b> ####

# make title more symetrically bigger not just bigger for fucks sake 
# move price to the right side, keep vol on left
# see if you can get a marker or military like custom google font one of the ones you like
# better yet use that program you wrote for sn logos actually to create your shit
# could put that Sans Peur Systems somewhere at least watermark it, maybe?
# key levels find up to 6?, scoring strategy?, then base allotment of position (1/3, 1/5) conditionally?
# we could add RSI down there, fuck could add ichimoku, do whatever with all this data access now

NOTES -> SMA Legend

In [56]:
# # SMA20, SMA50, and SMA200 has to be in 'dell_df'
# legend_info = {
#     'Indicator': ['SMA20', 'SMA50', 'SMA200'],
#     'Color': ['blue', 'orange', 'green'],
#     'Description': ['20-day Moving Average', '50-day Moving Average', '200-day Moving Average']
# }

# legend_df = pd.DataFrame(legend_info)
# print(legend_df)


NOTES -> custom mplfinance chart setup example

In [57]:
# """    
# Custom Theme setup
# """

# dark_style = mpf.make_mpf_style(base_mpf_style='charles', 
#                                 rc={'axes.facecolor': 'black',
#                                     'figure.facecolor': 'darkslategrey',
#                                     'axes.grid': True,
#                                     'grid.color': 'gray',
#                                     'grid.linestyle': '--',
#                                     'grid.linewidth': 0.6,
#                                     'axes.titlesize': 16,
#                                     'axes.titleweight': 'bold',
#                                     'axes.labelsize': 12,
#                                     'axes.labelweight': 'bold',
#                                     'axes.labelcolor': 'white',
#                                     'axes.edgecolor': 'white',
#                                     'axes.linewidth': 2,
#                                     'xtick.color': 'white',
#                                     'xtick.labelsize': 10,
#                                     'ytick.color': 'white',
#                                     'ytick.labelsize': 10,
#                                     'figure.titlesize': 18,
#                                     'figure.titleweight': 'bold',
#                                     'legend.facecolor': 'grey',
#                                     'legend.edgecolor': 'white',
#                                     'legend.fontsize': 10,
#                                     'legend.title_fontsize': 12})

# mpf.plot(strv_df, 
#          type='candle', 
#          style=dark_style,  # Using the custom dark style
#          volume=True, 
#          title='STRV Candlestick Chart',
#          mav=(20, 50, 200),  # Adding moving averages of 20, 50, and 200 periods
#          figratio=(15, 8), 
#          tight_layout=True)

In [58]:
# i need a math checker for trading days vs calendar days (252/365)

Notes -> yahoo finance API exaple (free) *with limitations

In [59]:
# import yfinance as yf
# from datetime import datetime, timedelta

In [60]:
# # yahoo finance data download
# start = datetime.now() - timedelta(days=60)
# end = datetime.now()
# yahoo_df = yf.download(gap_up_stocks_list, start, end)
# yahoo_df = yahoo_df.loc[:,'Close']
# # view the yahoo data yahoo finance formatting multi-index (has a 60day max with standard API maneuvers)
# yahoo_df.head(10)